In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb


import seaborn as sns
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
# load dataset
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
#from sklearn.cross_validation import  train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
#from sklearn.grid_search import GridSearchCV2 #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from catboost import CatBoostRegressor
# Any results you write to the current directory are saved as output.

In [2]:
#loading train and test data-set

train = pd.read_csv("file:///D:/data%20analysis/COMPETITIONS%202020/IDAO%202020/IDAO%202020/train.csv",header=0)
test = pd.read_csv("file:///D:/data%20analysis/COMPETITIONS%202020/IDAO%202020/IDAO%202020/Track%201/test.csv",header=0)
print(train.shape,test.shape)

(649912, 15) (284071, 9)


In [3]:


def date_time_extract(a,count):
    if count == 0:
        print('extracting date, time, hour......for train data')
        train['hour'] = pd.DatetimeIndex(a).hour
        train['minute'] = pd.DatetimeIndex(a).minute
        train['second'] = pd.DatetimeIndex(a).second
        train['day'] = pd.DatetimeIndex(a).day
        print('extraction of date_time_hour done.........!!')
    else:
        print('extracting date, time, hour......for test data')
        test['hour'] = pd.DatetimeIndex(a).hour
        test['minute'] = pd.DatetimeIndex(a).minute
        test['second'] = pd.DatetimeIndex(a).second
        test['day'] = pd.DatetimeIndex(a).day
        print('extraction of date_time_hour done.........!!')
    
    count+=1
    
    
    

date_time_extract(train['epoch'],0)
date_time_extract(test['epoch'],1)

extracting date, time, hour......for train data
extraction of date_time_hour done.........!!
extracting date, time, hour......for test data
extraction of date_time_hour done.........!!


In [4]:
def get_acceleration(a,b,c,count):
    a1 = []
    a2 = []
    a3 = []
    
    #a1.append(0)
    #a2.append(0)
    #a3.append(0)
    
    a1 = np.asarray(a)
    a2 = np.asarray(b)
    a3 = np.asarray(c)
    
    b1 = []
    b2 = []
    b3 = []
    
    if count == 0:
    
        for i in range(len(np.asarray(train['hour']))-1):
            #print('extracting acceleration in 3-D....')
            b1.append((a1[i+1]-a1[i]))
            b2.append((a2[i+1]-a2[i]))
            b3.append((a3[i+1]-a3[i]))
        
        b1.append(a[len(a)-1])
        b2.append(b[len(a)-1])
        b3.append(c[len(a)-1])
    
        print('extraction done!!')
    
        print('loading acceleration values in train...')
        train['a_x'] = np.asarray(b1)
        train['a_y'] = np.asarray(b2)
        train['a_z'] = np.asarray(b3)
        print('loaded acc values in train...')
    
    else:
        
        for i in range(len(np.asarray(test['hour']))-1):
            #print('extracting acceleration in 3-D....')
            b1.append((a1[i+1]-a1[i]))
            b2.append((a2[i+1]-a2[i]))
            b3.append((a3[i+1]-a3[i]))
        
        b1.append(a[len(a)-1])
        b2.append(b[len(a)-1])
        b3.append(c[len(a)-1])
        
        
        
        
        print('loading acceleration values in test')
        test['a_x'] = np.asarray(b1)
        test['a_y'] = np.asarray(b2)
        test['a_z'] = np.asarray(b3)
        print('loaded acc values in test')
    
    
        

In [5]:
get_acceleration(train['Vx_sim'],train['Vy_sim'],train['Vz_sim'],0)
get_acceleration(test['Vx_sim'],test['Vy_sim'],test['Vz_sim'],1)

extraction done!!
loading acceleration values in train...
loaded acc values in train...
loading acceleration values in test
loaded acc values in test


In [6]:
def get_features(t,count):
    if count == 0:
        feature_df_train = train[['Vx_sim', 'Vy_sim', 'Vz_sim', 'x_sim', 'y_sim', 'z_sim','a_x','a_y','a_z']]
        return feature_df_train
    
    else:
        feature_df_test = test[['Vx_sim', 'Vy_sim', 'Vz_sim', 'x_sim', 'y_sim', 'z_sim','a_x','a_y','a_z']]
        return feature_df_test
    

In [7]:
x_train = get_features(train,0)
x_train = np.asarray(x_train)
x_test = get_features(test,1)
x_test = np.asarray(x_test)

In [8]:
def get_target_variable(x):
    return np.asarray(x)


In [9]:
y_x = get_target_variable(train['x'])
y_y = get_target_variable(train['y'])
y_z = get_target_variable(train['z'])

y_vx = get_target_variable(train['Vx'])
y_vy = get_target_variable(train['Vy'])
y_vz = get_target_variable(train['Vz'])

In [10]:
x_train_split,x_val_split,y_train_x,y_val_x,y_train_y,y_val_y,y_train_z,y_val_z,y_train_vx, y_val_vx,y_train_vy,y_val_vy, y_train_vz,y_val_vz = train_test_split(x_train,y_x,y_y,y_z,y_vx,y_vy,y_vz,test_size = 0.3,random_state = 5)

In [11]:
##### defining the score

def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    beta = np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))
    return 100*(1-beta)

def Mean_abs_error(Y_true,Y_pred):
    
    return mean_absolute_error(Y_true,Y_pred)


In [12]:
"""
# this is best till now
#n_estimators = 3000
#max_depth = 8

def xgb_model_evaluation(x_train_split,x_val_split,y_train_x,y_val_x,x_test):
    #n_Estimators = 10000
    model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.1,
                 max_depth=8,
                 min_child_weight=1.5,
                 n_estimators=3000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
    #print('all parameters enabled--->')
    print('train for y_train_x !!------>')
    
    model.fit(x_train_split,y_train_x)
    
    print('training for y_train_x completed-------->prediction started for y_val_x')
    
    a = model.predict(x_val_split)
    
    print('prediction for y_val_x completed------> evaluating result for validation data')
    
    print('the score for validation obtained is:',smape(a,y_val_x))
    print('the Mean_abs_error VALIDATION for train is:', Mean_abs_error(a,y_val_x))
    
    
    print('evaluating results for train---->')
    a = model.predict(x_train_split)
    print('the smape score for train obtained is:',smape(a,y_train_x))
    print('the Mean_abs_error obtained for train is:', Mean_abs_error(a,y_train_x))
    print('prediction of test has been started')
    
    a = np.asarray(model.predict(x_test))
    print('test values predicted successfully------> Moving to the next step now')
    return a,model
    
    
    

"""

"\n# this is best till now\n#n_estimators = 3000\n#max_depth = 8\n\ndef xgb_model_evaluation(x_train_split,x_val_split,y_train_x,y_val_x,x_test):\n    #n_Estimators = 10000\n    model = xgboost.XGBRegressor(colsample_bytree=0.4,\n                 gamma=0,                 \n                 learning_rate=0.1,\n                 max_depth=8,\n                 min_child_weight=1.5,\n                 n_estimators=3000,                                                                    \n                 reg_alpha=0.75,\n                 reg_lambda=0.45,\n                 subsample=0.6,\n                 seed=42)\n    #print('all parameters enabled--->')\n    print('train for y_train_x !!------>')\n    \n    model.fit(x_train_split,y_train_x)\n    \n    print('training for y_train_x completed-------->prediction started for y_val_x')\n    \n    a = model.predict(x_val_split)\n    \n    print('prediction for y_val_x completed------> evaluating result for validation data')\n    \n    print('the

In [13]:
# this is best till now
#n_estimators = 10000
#num_leaves = 80
#learning_rate = 0.05

def lgb_model_evaluation(x_train_split,x_val_split,y_train_x,y_val_x,x_test):
    #n_Estimators = 10000
    #LGBMRegressor
    model = lgb.LGBMRegressor(objective='regression',
    num_leaves=100,
    learning_rate=0.1,
    n_estimators=15000)
    
    
    #print('all parameters enabled--->')
    print('train for y_train_x !!------>')
    
    model.fit(x_train_split,y_train_x)
    
    print('training for y_train_x completed-------->prediction started for y_val_x')
    
    a = model.predict(x_val_split)
    
    print('prediction for y_val_x completed------> evaluating result for validation data')
    
    print('the score for validation obtained is:',smape(a,y_val_x))
    print('the Mean_abs_error VALIDATION for train is:', Mean_abs_error(a,y_val_x))
    
    
    print('evaluating results for train---->')
    a = model.predict(x_train_split)
    print('the smape score for train obtained is:',smape(a,y_train_x))
    print('the Mean_abs_error obtained for train is:', Mean_abs_error(a,y_train_x))
    print('prediction of test has been started')
    
    a = np.asarray(model.predict(x_test))
    print('test values predicted successfully------> Moving to the next step now')
    return a,model
    
    

In [14]:
# xgb model for y_train_x
y_test_x, model_x = lgb_model_evaluation(x_train_split,x_val_split,y_train_x,y_val_x,x_test)

train for y_train_x !!------>
training for y_train_x completed-------->prediction started for y_val_x
prediction for y_val_x completed------> evaluating result for validation data
the score for validation obtained is: 95.52708763028306
the Mean_abs_error VALIDATION for train is: 724.5130116996994
evaluating results for train---->
the smape score for train obtained is: 96.53945754848735
the Mean_abs_error obtained for train is: 483.98537046714284
prediction of test has been started
test values predicted successfully------> Moving to the next step now


In [15]:
# xgb model for y_train_y
y_test_y, model_y =lgb_model_evaluation(x_train_split,x_val_split,y_train_y,y_val_y,x_test)

train for y_train_x !!------>
training for y_train_x completed-------->prediction started for y_val_x
prediction for y_val_x completed------> evaluating result for validation data
the score for validation obtained is: 95.33226166666987
the Mean_abs_error VALIDATION for train is: 754.9813383445877
evaluating results for train---->
the smape score for train obtained is: 96.38282465293541
the Mean_abs_error obtained for train is: 493.96901342095333
prediction of test has been started
test values predicted successfully------> Moving to the next step now


In [16]:
# xgb model for y_train_x
y_test_z, model_z = lgb_model_evaluation(x_train_split,x_val_split,y_train_z,y_val_z,x_test)

train for y_train_x !!------>
training for y_train_x completed-------->prediction started for y_val_x
prediction for y_val_x completed------> evaluating result for validation data
the score for validation obtained is: 94.57048949939811
the Mean_abs_error VALIDATION for train is: 555.2353672965229
evaluating results for train---->
the smape score for train obtained is: 95.63759833432546
the Mean_abs_error obtained for train is: 371.13239698325253
prediction of test has been started
test values predicted successfully------> Moving to the next step now


In [17]:
# xgb model for y_train_vx
y_test_vx, model_vx = lgb_model_evaluation(x_train_split,x_val_split,y_train_vx,y_val_vx,x_test)

train for y_train_x !!------>
training for y_train_x completed-------->prediction started for y_val_x
prediction for y_val_x completed------> evaluating result for validation data
the score for validation obtained is: 95.53128680876983
the Mean_abs_error VALIDATION for train is: 0.07217995822414498
evaluating results for train---->
the smape score for train obtained is: 96.58181981190091
the Mean_abs_error obtained for train is: 0.049748451747584466
prediction of test has been started
test values predicted successfully------> Moving to the next step now


In [18]:
# xgb model for y_train_vy
y_test_vy, model_vy = lgb_model_evaluation(x_train_split,x_val_split,y_train_vy,y_val_vy,x_test)

train for y_train_x !!------>
training for y_train_x completed-------->prediction started for y_val_x
prediction for y_val_x completed------> evaluating result for validation data
the score for validation obtained is: 95.4764360361155
the Mean_abs_error VALIDATION for train is: 0.0742793874131463
evaluating results for train---->
the smape score for train obtained is: 96.47635722896962
the Mean_abs_error obtained for train is: 0.051523038516330034
prediction of test has been started
test values predicted successfully------> Moving to the next step now


In [19]:
# xgb model for y_train_vz
y_test_vz, model_vz = lgb_model_evaluation(x_train_split,x_val_split,y_train_vz,y_val_vz,x_test)

train for y_train_x !!------>
training for y_train_x completed-------->prediction started for y_val_x
prediction for y_val_x completed------> evaluating result for validation data
the score for validation obtained is: 94.56553171669574
the Mean_abs_error VALIDATION for train is: 0.05562930803806017
evaluating results for train---->
the smape score for train obtained is: 95.72860059690234
the Mean_abs_error obtained for train is: 0.039489822826401406
prediction of test has been started
test values predicted successfully------> Moving to the next step now


In [20]:
y_test_vx.shape

(284071,)

In [21]:
x_test.shape

(284071, 9)

In [22]:
import pandas as pd
#test = pd.read_csv("test.csv")

# just sending simulated values as the answer
#submission = test[["id"]]
#test = pd.read_csv("/content/drive/My Drive/IDAO 2020/IDAO 2020/Track 1/test.csv",header=0)
test = pd.read_csv("file:///D:/data%20analysis/COMPETITIONS%202020/IDAO%202020/IDAO%202020/Track%201/test.csv",header=0)
test = test.drop('sat_id',axis=1)
test = test.drop('epoch',axis=1)

idd = np.asarray(test['id'])



submissio = test[["id", "x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"]]
submissio.columns = ["id", "x", "y", "z", "Vx", "Vy", "Vz"]
test = test[:0]

submissio['id'] = idd
submissio['x'] = np.asarray(y_test_x)
submissio['y'] = np.asarray(y_test_y)
submissio['z'] = np.asarray(y_test_z)

submissio['Vx'] = np.asarray(y_test_vx)
submissio['Vy'] = np.asarray(y_test_vy)
submissio['Vz'] = np.asarray(y_test_vz)

submissio.to_csv("D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\submission.csv", index=False)

In [23]:
########## saving our model to D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\

import pickle 

from sklearn.externals import joblib 
# Save the trained model as a pickle string. 

print('saving our model to D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\attempt 1')

model_x_saved = joblib.dump(model_x, 'D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\model_x.pkl') 
model_y_saved = joblib.dump(model_y, 'D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\model_y.pkl') 
model_z_saved = joblib.dump(model_z, 'D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\model_z.pkl') 
    
model_vx_saved = joblib.dump(model_vx, 'D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\model_vx.pkl') 
model_vy_saved = joblib.dump(model_vy, 'D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\model_vy.pkl') 
model_vz_saved = joblib.dump(model_vz, 'D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissions\model_vz.pkl') 

print('models saved-----> now they are ready to be used')

"""
# Load the model from the file 
knn_from_joblib = joblib.load('filename.pkl')  
  
# Use the loaded model to make predictions 
knn_from_joblib.predict(X_test)
"""

C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


saving our model to D:\data analysis\COMPETITIONS 2020\IDAO 2020\IDAO 2020\Track 2\my submissionsttempt 1
models saved-----> now they are ready to be used


"\n# Load the model from the file \nknn_from_joblib = joblib.load('filename.pkl')  \n  \n# Use the loaded model to make predictions \nknn_from_joblib.predict(X_test)\n"

In [24]:
##loading the saved model

"""
from sklearn.externals import joblib 
model_x = joblib.load('D:\\data analysis\\COMPETITIONS 2020\\IDAO 2020\\IDAO 2020\\Track 2\\my submissions\\attempt 1\\n_es = 3000, xgb\\model_x.pkl')


x_test = test[['Vx_sim', 'Vy_sim', 'Vz_sim', 'x_sim', 'y_sim', 'z_sim','a_x','a_y','a_z']]

model_x.predict(np.asarray(x_test[:150]))
"""

"\nfrom sklearn.externals import joblib \nmodel_x = joblib.load('D:\\data analysis\\COMPETITIONS 2020\\IDAO 2020\\IDAO 2020\\Track 2\\my submissions\\attempt 1\\n_es = 3000, xgb\\model_x.pkl')\n\n\nx_test = test[['Vx_sim', 'Vy_sim', 'Vz_sim', 'x_sim', 'y_sim', 'z_sim','a_x','a_y','a_z']]\n\nmodel_x.predict(np.asarray(x_test[:150]))\n"

In [25]:
#test['a_x']